<h4>Loading the Comments</h4>

In [1]:
import pandas as pd

comments = pd.read_csv("comments.csv")
comments.head()

,rate,comment
0,5.0,در مقایسه آی‌پد ۲۰۲۲( نسل ۱۰) و مینی نسل ۶، هر...
1,5.0,بسیار زیبا و مناسب برای طراحی و کارای روز مره ...
2,5.0,عالیه حرف نداره بازی ها هم خیلی خوب اجرا میکنه
3,5.0,جنس بدنه با کیفیت نرم افزار روان و بی نقص در ...
4,5.0,کالا اصل بود


<h4>Normalizing</h4>

In [2]:
import re
from hazm import Normalizer


def map_replacer(text, mapping):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))


def hazm_normalizer(text):
    return Normalizer().normalize(text)


def rate_normalizer(rate):
    if rate < 2.75:
        rate = -1
    elif rate > 3.5:
        rate = 1
    else:
        rate = 0
    return rate


def comment_normalizer(comment):
    if not isinstance(comment, str):
        return ""

    # Remove all special chars
    comment = re.sub(r'[\d+a-zA-Z\s{2,}<>#.:()"\'!?؟،,@$%^&*_+\[\]/]', " ", comment)

    # Replace arabic chars with persian
    comment = map_replacer(
        comment,
        {
            "إ": "ا",
            "أ": "ا",
            "ة": "ه",
            "ؤ": "و",
            "ك": "ک",
            "ى": "ی",
            "ي": "ی",
            "ئ": "ی",
        },
    )
    

    comment = comment.strip()
    return comment


comments["rate"] = comments["rate"].apply(rate_normalizer)
comments["comment"] = comments["comment"].apply(comment_normalizer)
comments = comments.dropna()
comments.to_csv("comments.normalized.csv", index=False)
comments.head()

,rate,comment
0,1,در مقایسه آی‌پد نسل و مینی نسل هر...
1,1,بسیار زیبا و مناسب برای طراحی و کارای روز مره ...
2,1,عالیه حرف نداره بازی ها هم خیلی خوب اجرا میکنه
3,1,جنس بدنه با کیفیت نرم افزار روان و بی نقص در ...
4,1,کالا اصل بود


In [3]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144717 entries, 0 to 144716
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   rate     144717 non-null  int64 
 1   comment  144717 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.2+ MB
